## Ingeniería de Big Data | Unidad 3 - Actividad 1 | UNICARIBE
- Detecta productos emergentes por categoría analizando tendencias de consumo en e-commerce.

In [14]:
# ============================
#     Carga de Datos
# ============================

import pandas as pd

# Ruta al dataset existente
data_path = "../data/dataset.csv"

# Cargar datos
df = pd.read_csv(data_path, parse_dates=["fecha"])
print("Datos cargados:", df.shape)

df.head()

Datos cargados: (50, 5)


,fecha,categoria,producto,unidades_vendidas,precio_unitario
0,2024-10-08,Hogar y Cocina,Cafetera,69,403.37
1,2024-02-05,Libros,Libro de Cocina,28,130.23
2,2024-11-12,Belleza,Sérum Facial,37,222.86
3,2024-12-30,Belleza,Secador de Pelo,5,75.92
4,2024-09-30,Ropa,Vestido de Verano,100,320.74


In [15]:
# ============================
#     Limpieza de Datos
# ============================

# Eliminar registros con valores nulos
df.dropna(inplace=True)

# Filtrar ventas válidas (>0)
df = df[df['unidades_vendidas'] > 0]

# Calcular ingresos
df['ingresos'] = df['unidades_vendidas'] * df['precio_unitario']

print("Datos después de limpieza:", df.shape)
df.head()

Datos después de limpieza: (50, 6)


,fecha,categoria,producto,unidades_vendidas,precio_unitario,ingresos
0,2024-10-08,Hogar y Cocina,Cafetera,69,403.37,27832.53
1,2024-02-05,Libros,Libro de Cocina,28,130.23,3646.44
2,2024-11-12,Belleza,Sérum Facial,37,222.86,8245.82
3,2024-12-30,Belleza,Secador de Pelo,5,75.92,379.60
4,2024-09-30,Ropa,Vestido de Verano,100,320.74,32074.00


In [16]:
# ============================
# Paso 3 - Detección de Productos Emergentes
# ============================

# Extraer mes
df['mes'] = df['fecha'].dt.to_period('M')

# Calcular ventas mensuales
ventas_mensuales = df.groupby(['mes', 'categoria', 'producto'])['unidades_vendidas'].sum().reset_index()

# Calcular crecimiento porcentual mes a mes
ventas_mensuales['crecimiento'] = ventas_mensuales.groupby(['categoria', 'producto'])['unidades_vendidas'].pct_change()

# Seleccionar últimos 3 meses
ultimos_meses = ventas_mensuales['mes'].sort_values().unique()[-3:]
emergentes = ventas_mensuales[ventas_mensuales['mes'].isin(ultimos_meses)]

# Promedio de crecimiento por producto
emergentes = emergentes.groupby(['categoria', 'producto'])['crecimiento'].mean().reset_index()

# Ordenar por crecimiento descendente
emergentes = emergentes.sort_values(by='crecimiento', ascending=False)

# Filtrar productos con crecimiento > 30%
emergentes = emergentes[emergentes['crecimiento'] > 0.3]

# Guardar resultados
df.to_csv("../powerbi/viz_dataset.csv", index=False)
emergentes.to_csv("../powerbi/viz_emergentes.csv", index=False)

print("Exportacion lista en carpeta 'powerbi/'")
emergentes.head()

Exportacion lista en carpeta 'powerbi/'


,categoria,producto,crecimiento
0,Electrónica,Reloj Inteligente Z,6.428571
4,Ropa,Camiseta,5.142857
1,Hogar y Cocina,Freidora de Aire,0.460000
